In [1]:
#Import der benötigten Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as skp
import tensorflow as tf
from tensorflow import keras as k
import math

In [2]:
os_dir = os.getcwd()
path_to_soundfilefolder = f'{os_dir}\\Datensatz\\processed_audio'
print(path_to_soundfilefolder)

E:\Notebooks\Genre_Classification\Datensatz\processed_audio


In [ ]:
#Zeug von Librosa
#y_harmonic, y_percussive = librosa.effects.hpss(y)  -------> noch beschreiben, was die einzelnen Attribute aussagen
#tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
#mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
#mfcc_delta = librosa.feature.delta(mfcc)
#beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)
#chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
#beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)
#beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [86]:
#Import der Sound-Dateien\n
import os
import numpy as np

#Zuordnung der Labels zu Zahlen
genre_names = []

#enthält sämtliche Einträge
features = []
#enthält die Ergebnisse sämtlicher Einträge
genre = []

print(path_to_soundfilefolder)

for filename in os.listdir(path_to_soundfilefolder):
    #hier kommen alle Attribute pro Soundfile rein
    attributes = []
    
    #Genre einfügen
    attributes.append(filename.split("_")[0])
    
    #Umwandlung der Soundfiles in repräsentative Zahlenwerte
    print(f"Converting: {filename}")
    y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\' + filename)
    
    #extrahieren der aussagekräftigen Eigenschaften
    stft_array = librosa.feature.chroma_stft(y=y, sr=sr)
    stft_mean = np.mean(stft_array)
    stft_var = np.var(stft_array)

    cq_array = librosa.feature.chroma_cqt(y=y, sr=sr)
    cq_mean = np.mean(cq_array)
    cq_var = np.var(cq_array)

    rms_array = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms_array)
    rms_var = np.var(rms_array)

    cent_array = librosa.feature.spectral_centroid(y=y, sr=sr)
    cent_mean = np.mean(cent_array)
    cent_var = np.var(cent_array)

    spec_array = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_mean = np.mean(spec_array)
    spec_var = np.var(spec_array)

    rolloff_array = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = np.mean(rolloff_array)
    rolloff_var = np.var(rolloff_array)

    zero_crossing_rate_array = librosa.feature.zero_crossing_rate(y)
    zero_crossing_rate_mean = np.mean(zero_crossing_rate_array)
    zero_crossing_rate_var = np.var(zero_crossing_rate_array)

    tonnetz_array = librosa.feature.tonnetz(y=y, sr=sr)
    tonnetz_mean = np.mean(tonnetz_array)
    tonnetz_var = np.var(tonnetz_array)

    y_harmonic_array, y_percussive = librosa.effects.hpss(y)
    tempo1, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
    tempo = np.float32(tempo1)
    
    y_harmonic_mean = np.mean(y_harmonic_array)
    y_harmonic_var = np.var(y_harmonic_array)

    attributes.extend([stft_mean, stft_var, cq_mean, cq_var, rms_mean, rms_var, cent_mean, cent_var, spec_mean, spec_var, rolloff_mean, rolloff_var, zero_crossing_rate_mean, zero_crossing_rate_var, tonnetz_mean, tonnetz_var, tempo, y_harmonic_mean, y_harmonic_var]) 

    #das mfcc-array besteht aus 20 arrays, für die jeweils mean und var bestimmt werden
    mfcc_array = librosa.feature.mfcc(y=y, sr=sr)
    for i in range(20):
        mfcc_mean = np.mean(mfcc_array[i])
        mfcc_var = np.var(mfcc_array[i])
        attributes.extend([mfcc_mean, mfcc_var])  
        
    
    
    #Anhängen eines Eintrages
    features.append(attributes)
    
    

E:\Notebooks\Genre_Classification\Datensatz\processed_audio
Converting: blues_100_0.wav
Converting: blues_100_1.wav
Converting: blues_100_2.wav
Converting: blues_100_3.wav
Converting: blues_100_4.wav
Converting: blues_100_5.wav
Converting: blues_100_6.wav
Converting: blues_100_7.wav
Converting: blues_100_8.wav
Converting: blues_100_9.wav
Converting: blues_10_0.wav
Converting: blues_10_1.wav
Converting: blues_10_2.wav
Converting: blues_10_3.wav
Converting: blues_10_4.wav
Converting: blues_10_5.wav
Converting: blues_10_6.wav
Converting: blues_10_7.wav
Converting: blues_10_8.wav
Converting: blues_10_9.wav
Converting: blues_11_0.wav
Converting: blues_11_1.wav
Converting: blues_11_2.wav
Converting: blues_11_3.wav
Converting: blues_11_4.wav
Converting: blues_11_5.wav
Converting: blues_11_6.wav
Converting: blues_11_7.wav
Converting: blues_11_8.wav
Converting: blues_11_9.wav


KeyboardInterrupt: 

In [87]:
#Array an konvertierten Eigenschaften in CSV speichern
arr = features # Hier muss dann das Ergebnis Array rein
headers = ["genre","stft_mean", "stft_var", "cq_mean", "cq_var", "rms_mean", "rms_var", "cent_mean", "cent_var", "spec_mean", "spec_var", "rolloff_mean", "rolloff_var", "zero_crossing_rate_mean", "zero_crossing_rate_var", "tonnetz_mean", "tonnetz_var", "tempo", "y_harmonic_mean", "y_harmonic_var", "mfcc_mean_1", "mfcc_var_1", "mfcc_mean_2", "mfcc_var_2", "mfcc_mean_3", "mfcc_var_3", "mfcc_mean_4", "mfcc_var_4", "mfcc_mean_5", "mfcc_var_5", "mfcc_mean_6", "mfcc_var_6", "mfcc_mean_7", "mfcc_var_7", "mfcc_mean_8", "mfcc_var_8", "mfcc_mean_9", "mfcc_var_9", "mfcc_mean_10", "mfcc_var_10", "mfcc_mean_11", "mfcc_var_11", "mfcc_mean_12", "mfcc_var_12", "mfcc_mean_13", "mfcc_var_13", "mfcc_mean_14", "mfcc_var_14", "mfcc_mean_15", "mfcc_var_15", "mfcc_mean_16", "mfcc_var_16", "mfcc_mean_17", "mfcc_var_17", "mfcc_mean_18", "mfcc_var_18", "mfcc_mean_19", "mfcc_var_19", "mfcc_mean_20", "mfcc_var_20"] # Das sind die Überschriften
df = pd.DataFrame(arr) 
pd.DataFrame(df).to_csv(f"{os_dir}\Datensatz\\results.csv", header = headers, index=False)

In [83]:
test_frame = pd.DataFrame(features) 

In [84]:
print(test_frame)

            0         1         2         3         4         5            6   \
0     0.430700  0.079948  0.443360  0.068219  0.056834  0.000283  1189.600872   
1     0.401774  0.090328  0.434674  0.064693  0.121461  0.005827  1545.366511   
2     0.346437  0.092592  0.447413  0.059083  0.133687  0.003669  1510.988881   
3     0.336843  0.078507  0.454218  0.071784  0.089527  0.003820  1576.137830   
4     0.360834  0.079291  0.464878  0.071757  0.070490  0.002473  1204.764662   
...        ...       ...       ...       ...       ...       ...          ...   
9977  0.372667  0.082638  0.262808  0.070318  0.057897  0.000088  1847.993567   
9978  0.347207  0.088840  0.284377  0.073732  0.052402  0.000701  1346.166434   
9979  0.387354  0.084762  0.248847  0.066627  0.066430  0.000320  2084.876439   
9980  0.369244  0.086697  0.291710  0.071916  0.050524  0.000067  1634.338774   
9981       NaN       NaN       NaN       NaN       NaN       NaN          NaN   

                 7         

In [69]:
df = pd.read_csv(f'Datensatz//results.csv')

data = df.iloc[0:, 1:]
song_labels = data['genre']
label_index = dict()

for i, x in enumerate(df.label.unique()):
    label_index[x] = i

labels = []

#print(len(song_labels))
for x in range(len(song_labels)):
    labels.append(label_index[song_labels[x]])

genre_list = pd.DataFrame(labels)

In [70]:
# normalize
cols = features.columns
min_max_scaler = skp.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)
features_normalized = pd.DataFrame(np_scaled, columns = cols)


In [77]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(features_normalized,genre_list,test_size = 0.2,random_state = 42)

scaler = skp.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)


      length  chroma_stft_mean  chroma_stft_var  rms_mean   rms_var  \
0        0.0         -0.954024         0.755136 -0.846810 -0.553232   
1        0.0         -1.461998         0.699073 -0.384951  0.258831   
2        0.0          0.843479         1.517352  2.043859  2.069544   
3        0.0         -2.127091         0.570655 -1.241262 -0.694352   
4        0.0         -0.767909         1.456974 -0.461418  0.811581   
...      ...               ...              ...       ...       ...   
7987     0.0         -1.240592         1.141071 -0.477121  0.323693   
7988     0.0         -1.457300        -0.327524 -1.305664 -0.712068   
7989     0.0         -1.548909         0.915723 -0.683575 -0.104507   
7990     0.0         -0.153339         0.762706 -0.325390 -0.105628   
7991     0.0          0.503368        -0.456082  0.631459 -0.204434   

      spectral_centroid_mean  spectral_centroid_var  spectral_bandwidth_mean  \
0                   0.756545              -0.385804                

In [78]:
#Programmierung des Modells
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=300)
model.summary()

Epoch 1/300
250/250 [==============================] - 1s 755us/step - loss: 1.9283 - accuracy: 0.3010
Epoch 2/300
250/250 [==============================] - 0s 706us/step - loss: 1.0853 - accuracy: 0.6232
Epoch 3/300
250/250 [==============================] - 0s 702us/step - loss: 0.8417 - accuracy: 0.7129
Epoch 4/300
250/250 [==============================] - 0s 706us/step - loss: 0.7477 - accuracy: 0.7498
Epoch 5/300
250/250 [==============================] - 0s 743us/step - loss: 0.6382 - accuracy: 0.7791
Epoch 6/300
250/250 [==============================] - 0s 682us/step - loss: 0.5880 - accuracy: 0.8010
Epoch 7/300
250/250 [==============================] - 0s 690us/step - loss: 0.5552 - accuracy: 0.8139
Epoch 8/300
250/250 [==============================] - 0s 694us/step - loss: 0.5051 - accuracy: 0.8302
Epoch 9/300
250/250 [==============================] - 0s 686us/step - loss: 0.4570 - accuracy: 0.8461
Epoch 10/300
250/250 [==============================] - 0s 702us/step - l

250/250 [==============================] - 0s 678us/step - loss: 0.0164 - accuracy: 0.9961
Epoch 80/300
250/250 [==============================] - 0s 650us/step - loss: 0.0121 - accuracy: 0.9977
Epoch 81/300
250/250 [==============================] - 0s 670us/step - loss: 0.0135 - accuracy: 0.9970
Epoch 82/300
250/250 [==============================] - 0s 705us/step - loss: 0.0121 - accuracy: 0.9982
Epoch 83/300
250/250 [==============================] - 0s 682us/step - loss: 0.0104 - accuracy: 0.9982
Epoch 84/300
250/250 [==============================] - 0s 731us/step - loss: 0.0217 - accuracy: 0.9942
Epoch 85/300
250/250 [==============================] - 0s 678us/step - loss: 0.0352 - accuracy: 0.9880
Epoch 86/300
250/250 [==============================] - 0s 678us/step - loss: 0.0661 - accuracy: 0.9760
Epoch 87/300
250/250 [==============================] - 0s 718us/step - loss: 0.0381 - accuracy: 0.9870
Epoch 88/300
250/250 [==============================] - 0s 709us/step - loss:

250/250 [==============================] - 0s 662us/step - loss: 0.0021 - accuracy: 0.9996
Epoch 158/300
250/250 [==============================] - 0s 654us/step - loss: 0.0101 - accuracy: 0.9978
Epoch 159/300
250/250 [==============================] - 0s 657us/step - loss: 0.0067 - accuracy: 0.9980
Epoch 160/300
250/250 [==============================] - 0s 646us/step - loss: 0.0629 - accuracy: 0.9816
Epoch 161/300
250/250 [==============================] - 0s 658us/step - loss: 0.0727 - accuracy: 0.9749
Epoch 162/300
250/250 [==============================] - 0s 658us/step - loss: 0.0114 - accuracy: 0.9966
Epoch 163/300
250/250 [==============================] - 0s 654us/step - loss: 0.0083 - accuracy: 0.9978
Epoch 164/300
250/250 [==============================] - 0s 666us/step - loss: 0.0054 - accuracy: 0.9988
Epoch 165/300
250/250 [==============================] - 0s 661us/step - loss: 0.0042 - accuracy: 0.9992
Epoch 166/300
250/250 [==============================] - 0s 654us/ste

250/250 [==============================] - 0s 662us/step - loss: 0.0033 - accuracy: 0.9990
Epoch 235/300
250/250 [==============================] - 0s 649us/step - loss: 0.0054 - accuracy: 0.9991
Epoch 236/300
250/250 [==============================] - 0s 658us/step - loss: 0.0043 - accuracy: 0.9988
Epoch 237/300
250/250 [==============================] - 0s 658us/step - loss: 0.0070 - accuracy: 0.9980
Epoch 238/300
250/250 [==============================] - 0s 662us/step - loss: 0.0121 - accuracy: 0.9960
Epoch 239/300
250/250 [==============================] - 0s 662us/step - loss: 0.1092 - accuracy: 0.9667
Epoch 240/300
250/250 [==============================] - 0s 662us/step - loss: 0.0421 - accuracy: 0.9874
Epoch 241/300
250/250 [==============================] - 0s 662us/step - loss: 0.0065 - accuracy: 0.9980
Epoch 242/300
250/250 [==============================] - 0s 927us/step - loss: 0.0048 - accuracy: 0.9988
Epoch 243/300
250/250 [==============================] - 0s 684us/ste

In [ ]:
#bestes Modell aus dem Internet
model1 = k.models.Sequential([
    k.layers.Dense(1024, activation='relu', input_shape=(x_train.shape[1],)),
    k.layers.Dropout(0.3),
    
    k.layers.Dense(512, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(10, activation='softmax'),
])
print(model1.summary())

model1.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')

model1.fit(x_train, y_train, epochs=300)

In [ ]:
import numpy as np
import sklearn.metrics

predictions_confidences = model.predict(x_test)

predictions = []
for entry in predictions_confidences:
        predictions.append(np.argmax(entry))
print(predictions)

In [ ]:
print(f"{sklearn.metrics.classification_report(y_test, predictions)}\n")

In [ ]:
#test librosa
y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\blues_1_0.wav')
print(y, sr)

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print(tempo, beat_frames)